In [ ]:
import numpy as np
import time
import random
import matplotlib.pyplot as plt

# 1.a)
def matrix_multiply(A, B):
# A function that computes the multiplication of square matrices
    n = len(A)  # The dimension of the matrix
    C = [[0] * n for _ in range(n)]  # Create a matrix where the solution is stored
    for i in range(n):  # Loop over each row of A
        for j in range(n):  # Loop over each column of B
            for k in range(n):  # Multiply and sum the elements of A and B
                C[i][j] += A[i][k] * B[k][j]
    return C

def numpy_multiply(A, B):
# Matrix multiplication using numpy
    return np.dot(np.array(A), np.array(B))

def strassen_multiply(A, B):
# Recursive version of Strassen's algorithm
    n = len(A) # Find matrix size
    if n == 1: # For 1x1 matrices
        return [[A[0][0] * B[0][0]]]
    else:
        mid = n // 2 # Middle of the matrix to divide
        A11, A12, A21, A22 = subdivide_matrix(A, mid) # Subdivide matrix A
        B11, B12, B21, B22 = subdivide_matrix(B, mid) # Subdivide matrix B

        # Strassen's formula, uses recursive calls to make more subdivisions (until they all are 1x1)
        M1 = strassen_multiply(matrix_add(A11, A22), matrix_add(B11, B22))
        M2 = strassen_multiply(matrix_add(A21, A22), B11)
        M3 = strassen_multiply(A11, matrix_subtract(B12, B22))
        M4 = strassen_multiply(A22, matrix_subtract(B21, B11))
        M5 = strassen_multiply(matrix_add(A11, A12), B22)
        M6 = strassen_multiply(matrix_subtract(A21, A11), matrix_add(B11, B12))
        M7 = strassen_multiply(matrix_subtract(A12, A22), matrix_add(B21, B22))

        # Combining the results to get the final matrix
        C11 = matrix_add(matrix_subtract(matrix_add(M1, M4), M5), M7)
        C12 = matrix_add(M3, M5)
        C21 = matrix_add(M2, M4)
        C22 = matrix_add(matrix_subtract(matrix_add(M1, M3), M2), M6)

        return combine_matrices(C11, C12, C21, C22)

def strassen_multiply_non_recursive(A, B):
# Non-recursive Strassen algorithm
    n = len(A)
    if n == 1:
        return [[A[0][0] * B[0][0]]]
    else:
        mid = n // 2 # Middle of the matrix to divide
        A11, A12, A21, A22 = subdivide_matrix(A, mid) 
        B11, B12, B21, B22 = subdivide_matrix(B, mid) 

        # Now uses numpy to be non-recursive
        M1 = numpy_multiply(matrix_add(A11, A22), matrix_add(B11, B22))
        M2 = numpy_multiply(matrix_add(A21, A22), B11)
        M3 = numpy_multiply(A11, matrix_subtract(B12, B22))
        M4 = numpy_multiply(A22, matrix_subtract(B21, B11))
        M5 = numpy_multiply(matrix_add(A11, A12), B22)
        M6 = numpy_multiply(matrix_subtract(A21, A11), matrix_add(B11, B12))
        M7 = numpy_multiply(matrix_subtract(A12, A22), matrix_add(B21, B22))

        # Combining results
        C11 = matrix_add(matrix_subtract(matrix_add(M1, M4), M5), M7)
        C12 = matrix_add(M3, M5)
        C21 = matrix_add(M2, M4)
        C22 = matrix_add(matrix_subtract(matrix_add(M1, M3), M2), M6)

        return combine_matrices(C11, C12, C21, C22)

# Helper functions
def subdivide_matrix(M, mid):
# Divides a matrix into 4 submatrices, starting with the first half of rows and columns    
    return [row[:mid] for row in M[:mid]], [row[mid:] for row in M[:mid]], \
           [row[:mid] for row in M[mid:]], [row[mid:] for row in M[mid:]]

def combine_matrices(A11, A12, A21, A22):
# Combines submatrices into 1 matrix
    top = [A11[i] + A12[i] for i in range(len(A11))] # Top half
    bottom = [A21[i] + A22[i] for i in range(len(A21))] # Bottom half
    return top + bottom # Combines top and bottom for the final result

def matrix_add(A, B):
# Elementwise addition
    return [[A[i][j] + B[i][j] for j in range(len(A[i]))] for i in range(len(A))]

def matrix_subtract(A, B):
# Elementwise subtraction
    return [[A[i][j] - B[i][j] for j in range(len(A[i]))] for i in range(len(A))]

def generate_matrix(n):
# Generates a nxn matrix containing random element from 0 to 10
    return [[random.randint(0, 10) for _ in range(n)] for _ in range(n)]

def benchmark(method, sizes):
# Benchmarks multiplication methods for different matrix sizes
    times = [] 
    for n in sizes:
        A = generate_matrix(n)
        B = generate_matrix(n)
        start_time = time.time()
        method(A, B)
        end_time = time.time()
        times.append(end_time - start_time)
    return times

# Define the sizes of matrices to test
sizes = [2**i for i in range(1, 11)]  # Powers of 2 from 2 to 1024

# Benchmarking all methods
naive_times = benchmark(matrix_multiply, sizes)
numpy_times = benchmark(numpy_multiply, sizes)
strassen_times = benchmark(strassen_multiply, sizes)
strassen_non_recursive_times = benchmark(strassen_multiply_non_recursive, sizes)

# Plotting the results
plt.figure(figsize=(10, 5))
plt.plot(sizes, naive_times, label='Naive Multiplication', marker='o')
plt.plot(sizes, numpy_times, label='NumPy Multiplication', marker='o')
plt.plot(sizes, strassen_times, label='Recursive Strassen', marker='o')
plt.plot(sizes, strassen_non_recursive_times, label='Non-Recursive Strassen', marker='o')
plt.xlabel('Matrix Size (n)')
plt.ylabel('Time (seconds)')
plt.title('Matrix Multiplication Timing Comparison')
plt.legend()
plt.grid(True)
plt.xscale('log', base=2)
plt.yscale('log')
plt.show()


# 1.d)
# After looking at the diagram, we can see that the naive implementation is faster with the
# calculation of a 2x2 matrix, since it uses simple python loops that can be processed quickly.
# NumPy could be less effective for those really small matrices, since it has to convert the lists
# into Numpy arrays first. But if the matrix gets bigger (f.e. 8x8), we can see that the naive approach
# takes more time, because it doesn't use any advanced algorithms. After we reach the 16x16 matrix
# NumPy is already around ten times more effective then the naive implementation.

# 1.e)
# Starting with 2x2 matrices, we can see that the non-recursive Strassen algorithm takes the most
# time, since it breaks down the matrices first and then uses NumPy, which converts the 1x1 matrices
# into NumPy arrays first. The recursive algorithm turns them into 1x1's no matter how big the matrix
# is and starts from the top of the function again. So instead of using NumPy it calculates the 1x1
# matrices like this: return [[A[0][0] * B[0][0]]], which makes it faster for the small matrices, 
# since it doesn't convert the lists into NumPy arrays. But on the long run, when the matrices get
# larger, we can see that the recursive algorithm takes the most time of all of them. That is like
# that because of the recursion, since it has to call the function again and again so many times. 
# With the 1024x1024 matrices, the non-recursive Strassen algorithm is as fast as NumPy. It actually
# differed with the tests because the outcomes were slightly different with each testing, but it always
# was either slightly below or above NumPy. That's because the non-recursive algorithm uses Numpy
# and Numpy also uses the Strassen algorithm for larger matrices. If we use our naive multiplication
# instead of NumPy in the non-recursive algorithm, NumPy is almost 100 times faster with the 2^10x2^10
# matrices and the non-recursive Strassen algorithm is slightly faster than the naive one, but probably
# will be much faster than the naive algorithm for matrices larger than that.

# Conclusion:
# The matrix multiplication of matrices larger than 10x10, will be much faster with NumPy, because it
# uses advanced algorithms for big matrices, like Strassen, and profits from the NumPy arrays.